In [424]:
!pip uninstall xlrd
!pip install xlrd

Found existing installation: xlrd 2.0.1
Uninstalling xlrd-2.0.1:
  Would remove:
    /usr/local/bin/runxlrd.py
    /usr/local/lib/python3.7/dist-packages/xlrd-2.0.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/xlrd/*
Proceed (y/n)? n
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [482]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [483]:
import pandas as pd
import numpy
import copy

In [512]:
siglas = pd.read_excel('gdrive/MyDrive/Hacka_Navi/siglas.xlsx')
siglas['ESTADO'] = siglas['ESTADO'].str.strip()
siglas_upper = siglas.copy(deep = True)
siglas_upper['ESTADO'] = siglas_upper['ESTADO'].str.upper()

In [513]:
siglas_upper.loc[0,'ESTADO'].strip()

'ACRE'

In [514]:
pop = pd.read_excel('gdrive/MyDrive/Hacka_Navi/PopulaçãoBrasileira_Municípios.xls')
pop1 = pop[~pop['POPULAÇÃO ESTIMADA'].str.contains('\(', na=False)]
pop2 = pop[pop['POPULAÇÃO ESTIMADA'].str.contains('\(', na=False)]
pop2['POPULAÇÃO ESTIMADA'] = pop2['POPULAÇÃO ESTIMADA'].str.replace(r"\(.*\)","")
pop2['POPULAÇÃO ESTIMADA'] = pop2['POPULAÇÃO ESTIMADA'].str.replace(r".","")
pop2['POPULAÇÃO ESTIMADA'] = pop2['POPULAÇÃO ESTIMADA'].str.strip()
pop1 = pop1.append(pop2)
print(len(pop1))
pop1.columns
pop1 = pop1.rename(columns={'UF':'ESTADO', 'NOME DO MUNICÍPIO':'MUNICIPIO', 'POPULAÇÃO ESTIMADA': 'POP'})
pop1['POP'] = pop1['POP'].astype(int)
pop = pop1

5570


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [515]:
pib_per_capita = pd.read_excel('gdrive/MyDrive/Hacka_Navi/pib_per_capita.xls')
pib_per_capita = pib_per_capita.rename(columns={'pib_per_capita':'PIB PER CAPITA'})
print(len(pib_per_capita))
pib_per_capita.columns

5570


Index(['ESTADO', 'MUNICIPIO', 'PIB PER CAPITA'], dtype='object')

In [516]:
solar = pd.read_csv('gdrive/MyDrive/Hacka_Navi/direct_normal_means_sedes-munic.csv', sep=';', encoding = 'utf-8-sig', decimal = ',')
solar.drop(['ID','LON','LAT','CLASS','JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'], axis = 1, inplace = True)
solar = solar.merge(siglas_upper, how = 'left',left_on='STATE', right_on='ESTADO')
solar.drop(['STATE', 'ESTADO'], axis = 1, inplace = True)
solar = solar.rename(columns={'NAME':'MUNICIPIO', 'ANNUAL':'SOL', 'SIGLA':'ESTADO'})
solar['ESTADO'] = solar['ESTADO'].str.strip()

In [517]:
distri = pd.read_csv('gdrive/MyDrive/Hacka_Navi/empreendimento-geracao-distribuida.csv', sep=';', encoding = 'utf-16', decimal = ',')
distri = distri[((distri['DscClasseConsumo'] == 'Residencial')|(distri['DscClasseConsumo'] == 'Comercial'))&(distri['DscFonteGeracao'] =='Radiação solar')]
#distri = distri[(distri['DscClasseConsumo'] == 'Residencial')&(distri['DscFonteGeracao'] =='Radiação solar')]
distri.drop(['DatGeracaoConjuntoDados', 'AnmPeriodoReferencia',
       'NumCNPJDistribuidora', 'SigAgente', 'NomAgente', 'CodClasseConsumo',
       'CodSubGrupoTarifario', 'DscSubGrupoTarifario',
       'codUFibge', 'codRegiao', 'NomRegiao', 'CodMunicipioIbge',
       'CodCEP', 'SigTipoConsumidor', 'NumCPFCNPJ',
       'NomeTitularEmpreendimento', 'CodEmpreendimento',
       'DthAtualizaCadastralEmpreend', 'SigModalidadeEmpreendimento',
       'DscModalidadeHabilitado', 'QtdUCRecebeCredito', 'SigTipoGeracao',
       'DscPorte', 'MdaPotenciaInstaladaKW','DscFonteGeracao',
       'NumCoordNEmpreendimento', 'NumCoordEEmpreendimento', 'NomSubEstacao',
       'NumCoordESub', 'NumCoordNSub'], axis = 1, inplace = True)
'''
distri = distri.groupby(['SigUF','NomMunicipio', 'DscClasseConsumo']).size().reset_index()
distri.drop(['DscClasseConsumo'], axis = 1, inplace = True)
distri = distri.rename(columns={'NomMunicipio':'MUNICIPIO', 'SigUF':'ESTADO', 0:'SOLAR_RESIDENCIAL'})
distri
'''

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (30) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


"\ndistri = distri.groupby(['SigUF','NomMunicipio', 'DscClasseConsumo']).size().reset_index()\ndistri.drop(['DscClasseConsumo'], axis = 1, inplace = True)\ndistri = distri.rename(columns={'NomMunicipio':'MUNICIPIO', 'SigUF':'ESTADO', 0:'SOLAR_RESIDENCIAL'})\ndistri\n"

In [518]:
distri_aux = distri.groupby(['SigUF','NomMunicipio', 'DscClasseConsumo']).size().reset_index()
#distri.drop(['DscClasseConsumo'], axis = 1, inplace = True)
#distri = distri.rename(columns={'NomMunicipio':'MUNICIPIO', 'SigUF':'ESTADO', 0:'SOLAR_RESIDENCIAL'})
distri1 = distri_aux[distri_aux['DscClasseConsumo']=='Residencial']
distri1.drop(['DscClasseConsumo'], axis = 1, inplace = True)
distri1 = distri1.rename(columns={'NomMunicipio':'MUNICIPIO', 'SigUF':'ESTADO', 0:'SOLAR_RESIDENCIAL'})
distri2 = distri_aux.groupby(['SigUF','NomMunicipio']).sum().reset_index()
distri2 = distri2.rename(columns={'NomMunicipio':'MUNICIPIO', 'SigUF':'ESTADO', 0:'TOTAL_USINAS'})
distri = distri1

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [519]:
casas = pd.read_excel('gdrive/MyDrive/Hacka_Navi/Casas_por_município.xlsx')
casas[['MUNICIPIO', 'ESTADO']] = casas['Brasil'].str.split('\(', 1, expand=True)
casas['ESTADO'] = casas['ESTADO'].str.replace('\)','')
casas['MUNICIPIO'] = casas['MUNICIPIO'].str.strip()
casas = casas.rename(columns={57324167:'CASAS'})
casas.drop('Brasil', axis = 1, inplace = True)
casas = casas.merge(distri, how = 'left',left_on=['ESTADO','MUNICIPIO'], right_on=['ESTADO','MUNICIPIO'])
casas = casas.fillna(value = 0)
casas['SOLAR_RESIDENCIAL'] = casas['SOLAR_RESIDENCIAL']/casas['CASAS']
print(len(casas))
casas.columns

5565


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


Index(['CASAS', 'MUNICIPIO', 'ESTADO', 'SOLAR_RESIDENCIAL'], dtype='object')

In [520]:
burocracia = pd.read_excel('gdrive/MyDrive/Hacka_Navi/Agilidade_Concessionária.xlsx')
print(len(burocracia))
burocracia.columns

27


Index(['ESTADO', 'INDICE DE BUROCRACIA'], dtype='object')

In [521]:
icms = pd.read_excel('gdrive/MyDrive/Hacka_Navi/Alíquota_ICMS_2022.xlsx')
icms['UF'] = icms['UF'].str.upper()
icms = icms.merge(siglas_upper, how = 'left',left_on='UF', right_on='ESTADO')
icms.drop(['UF', 'ESTADO'], axis = 1, inplace = True)
icms = icms.rename(columns={'SIGLA':'ESTADO','kWh':'ICMS'})
icms['ESTADO'] = icms['ESTADO'].str.strip()
print(len(icms))
icms.columns

27


Index(['ICMS', 'ESTADO'], dtype='object')

In [522]:
preco_kwh = pd.read_excel('gdrive/MyDrive/Hacka_Navi/PRECOKWH.xlsx')
print(len(preco_kwh))
preco_kwh.columns

27


Index(['ESTADO', 'REAIS_POR_KWH'], dtype='object')

In [523]:
preco_m2 = pd.read_excel('gdrive/MyDrive/Hacka_Navi/PRECOM2.xlsx')
preco_m2 = preco_m2.rename(columns={'UF':'ESTADO', 'NOME DO MUNICÍPIO':'MUNICIPIO', 'POPULAÇÃO ESTIMADA': 'POP'})
print(len(preco_m2))
preco_m2.columns

27


Index(['ESTADO', 'PRECO(REAIS/M2)'], dtype='object')

In [524]:
inadimplencia = pd.read_excel('gdrive/MyDrive/Hacka_Navi/TX_Inadimplência_2022.xlsx')
print(len(inadimplencia))
inadimplencia.columns

27


Index(['ESTADO', 'INADIMPLENCIA'], dtype='object')

In [525]:
dados_consolidados = pop.merge(pib_per_capita, how = 'left',left_on=['ESTADO','MUNICIPIO'], right_on=['ESTADO','MUNICIPIO'])\
                          .merge(solar, how = 'left',left_on=['ESTADO','MUNICIPIO'], right_on=['ESTADO','MUNICIPIO'])\
                          .merge(casas, how = 'left',left_on=['ESTADO','MUNICIPIO'], right_on=['ESTADO','MUNICIPIO'])\
                          .merge(burocracia, how = 'left',left_on=['ESTADO'], right_on=['ESTADO'])\
                          .merge(icms, how = 'left',left_on=['ESTADO'], right_on=['ESTADO'])\
                          .merge(preco_kwh, how = 'left',left_on=['ESTADO'], right_on=['ESTADO'])\
                          .merge(preco_m2, how = 'left',left_on=['ESTADO'], right_on=['ESTADO'])\
                          .merge(inadimplencia, how = 'left',left_on=['ESTADO'], right_on=['ESTADO'])
dados_consolidados

,ESTADO,MUNICIPIO,POP,PIB PER CAPITA,SOL,CASAS,SOLAR_RESIDENCIAL,INDICE DE BUROCRACIA,ICMS,REAIS_POR_KWH,PRECO(REAIS/M2),INADIMPLENCIA
0,RO,Alta Floresta D'Oeste,22516,21600.73,3387.0,7270.0,0.012242,55.5,0.185,0.54603,1559.93,0.4191
1,RO,Ariquemes,111148,23908.38,3067.0,27166.0,0.034933,55.5,0.185,0.54603,1559.93,0.4191
2,RO,Cabixi,5067,26347.78,3831.0,1975.0,0.011139,55.5,0.185,0.54603,1559.93,0.4191
3,RO,Cacoal,86416,26483.95,3374.0,24215.0,0.029321,55.5,0.185,0.54603,1559.93,0.4191
4,RO,Cerejeiras,16088,31051.55,3744.0,5348.0,0.014211,55.5,0.185,0.54603,1559.93,0.4191
...,...,...,...,...,...,...,...,...,...,...,...,...
5565,PE,Ferreiros,12216,9193.16,4774.0,3106.0,0.003220,51.0,0.270,0.73580,1421.44,0.4128
5566,BA,Coronel João Sá,15549,9240.57,4457.0,4868.0,0.000822,52.5,0.260,0.74949,1531.13,0.3683
5567,BA,Ibiassucê,8849,9549.91,5297.0,2798.0,0.008220,52.5,0.260,0.74949,1531.13,0.3683
5568,BA,Rodelas,9548,10436.49,5539.0,2094.0,0.008118,52.5,0.260,0.74949,1531.13,0.3683


In [526]:
dados_consolidados = dados_consolidados.merge(distri2, how = 'left',left_on=['ESTADO','MUNICIPIO'], right_on=['ESTADO','MUNICIPIO'])

In [527]:
dados_consolidados

,ESTADO,MUNICIPIO,POP,PIB PER CAPITA,SOL,CASAS,SOLAR_RESIDENCIAL,INDICE DE BUROCRACIA,ICMS,REAIS_POR_KWH,PRECO(REAIS/M2),INADIMPLENCIA,TOTAL_USINAS
0,RO,Alta Floresta D'Oeste,22516,21600.73,3387.0,7270.0,0.012242,55.5,0.185,0.54603,1559.93,0.4191,105.0
1,RO,Ariquemes,111148,23908.38,3067.0,27166.0,0.034933,55.5,0.185,0.54603,1559.93,0.4191,1047.0
2,RO,Cabixi,5067,26347.78,3831.0,1975.0,0.011139,55.5,0.185,0.54603,1559.93,0.4191,32.0
3,RO,Cacoal,86416,26483.95,3374.0,24215.0,0.029321,55.5,0.185,0.54603,1559.93,0.4191,787.0
4,RO,Cerejeiras,16088,31051.55,3744.0,5348.0,0.014211,55.5,0.185,0.54603,1559.93,0.4191,97.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,PE,Ferreiros,12216,9193.16,4774.0,3106.0,0.003220,51.0,0.270,0.73580,1421.44,0.4128,16.0
5566,BA,Coronel João Sá,15549,9240.57,4457.0,4868.0,0.000822,52.5,0.260,0.74949,1531.13,0.3683,6.0
5567,BA,Ibiassucê,8849,9549.91,5297.0,2798.0,0.008220,52.5,0.260,0.74949,1531.13,0.3683,29.0
5568,BA,Rodelas,9548,10436.49,5539.0,2094.0,0.008118,52.5,0.260,0.74949,1531.13,0.3683,27.0


In [530]:
#Tratar os dados agora
#dcn = dados consolidados normalizados
dcn = dados_consolidados.copy(deep=True)

dcn['POP_SCORE'] =(dcn['POP']-dcn['POP'].min())/(dcn['POP'].max()-dcn['POP'].min())
dcn['PIB_PER_CAPITA_SCORE'] =(dcn['PIB PER CAPITA']-dcn['PIB PER CAPITA'].min())/(dcn['PIB PER CAPITA'].max()-dcn['PIB PER CAPITA'].min())
dcn['SOL_SCORE'] =(dcn['SOL']-dcn['SOL'].min())/(dcn['SOL'].max()-dcn['SOL'].min())
dcn['CASAS_SCORE'] =(dcn['CASAS']-dcn['CASAS'].min())/(dcn['CASAS'].max()-dcn['CASAS'].min())
dcn['SOLAR_RESIDENCIAL_SCORE'] =(dcn['SOLAR_RESIDENCIAL']-dcn['SOLAR_RESIDENCIAL'].min())/(dcn['SOLAR_RESIDENCIAL'].max()-dcn['SOLAR_RESIDENCIAL'].min())
dcn['INDICE_DE_BUROCRACIA_SCORE'] =(dcn['INDICE DE BUROCRACIA']-dcn['INDICE DE BUROCRACIA'].min())/(dcn['INDICE DE BUROCRACIA'].max()-dcn['INDICE DE BUROCRACIA'].min())
dcn['ICMS_SCORE'] =(dcn['ICMS']-dcn['ICMS'].min())/(dcn['ICMS'].max()-dcn['ICMS'].min())
dcn['REAIS_POR_KWH_SCORE'] =(dcn['REAIS_POR_KWH']-dcn['REAIS_POR_KWH'].min())/(dcn['REAIS_POR_KWH'].max()-dcn['REAIS_POR_KWH'].min())
dcn['PRECO(REAIS/M2)_SCORE'] =(dcn['PRECO(REAIS/M2)']-dcn['PRECO(REAIS/M2)'].min())/(dcn['PRECO(REAIS/M2)'].max()-dcn['PRECO(REAIS/M2)'].min())
dcn['INADIMPLENCIA_SCORE'] =(dcn['INADIMPLENCIA']-dcn['INADIMPLENCIA'].min())/(dcn['INADIMPLENCIA'].max()-dcn['INADIMPLENCIA'].min())

In [562]:
dcn

,ESTADO,MUNICIPIO,POP,PIB PER CAPITA,SOL,CASAS,SOLAR_RESIDENCIAL,INDICE DE BUROCRACIA,ICMS,REAIS_POR_KWH,...,POP_SCORE,PIB_PER_CAPITA_SCORE,SOL_SCORE,CASAS_SCORE,SOLAR_RESIDENCIAL_SCORE,INDICE_DE_BUROCRACIA_SCORE,ICMS_SCORE,REAIS_POR_KWH_SCORE,PRECO(REAIS/M2)_SCORE,INADIMPLENCIA_SCORE
0,RO,Alta Floresta D'Oeste,22516,21600.73,3387.0,7270.0,0.012242,55.5,0.185,0.54603,...,0.001754,0.037180,0.285714,0.001967,0.023822,0.555556,0.258065,0.012850,0.487804,0.457881
1,RO,Ariquemes,111148,23908.38,3067.0,27166.0,0.034933,55.5,0.185,0.54603,...,0.008905,0.042193,0.207436,0.007534,0.067976,0.555556,0.258065,0.012850,0.487804,0.457881
2,RO,Cabixi,5067,26347.78,3831.0,1975.0,0.011139,55.5,0.185,0.54603,...,0.000347,0.047491,0.394325,0.000486,0.021676,0.555556,0.258065,0.012850,0.487804,0.457881
3,RO,Cacoal,86416,26483.95,3374.0,24215.0,0.029321,55.5,0.185,0.54603,...,0.006909,0.047787,0.282534,0.006708,0.057054,0.555556,0.258065,0.012850,0.487804,0.457881
4,RO,Cerejeiras,16088,31051.55,3744.0,5348.0,0.014211,55.5,0.185,0.54603,...,0.001236,0.057708,0.373043,0.001429,0.027653,0.555556,0.258065,0.012850,0.487804,0.457881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,PE,Ferreiros,12216,9193.16,4774.0,3106.0,0.003220,51.0,0.270,0.73580,...,0.000923,0.010231,0.625000,0.000802,0.006265,0.000000,0.806452,0.404899,0.113304,0.425323
5566,BA,Coronel João Sá,15549,9240.57,4457.0,4868.0,0.000822,52.5,0.260,0.74949,...,0.001192,0.010334,0.547456,0.001295,0.001599,0.185185,0.741935,0.433181,0.409924,0.195349
5567,BA,Ibiassucê,8849,9549.91,5297.0,2798.0,0.008220,52.5,0.260,0.74949,...,0.000652,0.011006,0.752935,0.000716,0.015995,0.185185,0.741935,0.433181,0.409924,0.195349
5568,BA,Rodelas,9548,10436.49,5539.0,2094.0,0.008118,52.5,0.260,0.74949,...,0.000708,0.012931,0.812133,0.000519,0.015797,0.185185,0.741935,0.433181,0.409924,0.195349


In [412]:
peso_pop = 2
peso_pib = 3
peso_sol = 5
peso_casas = 2
peso_pen = 3
peso_buro = 1
peso_icms = 1
peso_preco_en = 2
peso_preco_maodeobra = 1
peso_inadim = 1

In [413]:
dcn['GD_SCORE'] = (dcn['POP_SCORE']*peso_pop \
                  + dcn['PIB_PER_CAPITA_SCORE']*peso_pib \
                  + dcn['SOL_SCORE']*peso_sol \
                  + dcn['CASAS_SCORE']*peso_casas \
                  + dcn['SOLAR_RESIDENCIAL_SCORE']*peso_pen \
                  + dcn['INDICE_DE_BUROCRACIA_SCORE']*peso_buro \
                  + dcn['ICMS_SCORE']*peso_icms \
                  + dcn['REAIS_POR_KWH_SCORE']*peso_preco_en \
                  + dcn['PRECO(REAIS/M2)_SCORE']*peso_preco_maodeobra \
                  + dcn['INADIMPLENCIA_SCORE']*peso_inadim) \
                  / (dcn['POP_SCORE'] \
                  + dcn['PIB_PER_CAPITA_SCORE'] \
                  + dcn['SOL_SCORE']
                  + dcn['CASAS_SCORE'] \
                  + dcn['SOLAR_RESIDENCIAL_SCORE'] \
                  + dcn['INDICE_DE_BUROCRACIA_SCORE'] \
                  + dcn['ICMS_SCORE'] \
                  + dcn['REAIS_POR_KWH_SCORE'] \
                  + dcn['PRECO(REAIS/M2)_SCORE'] \
                  + dcn['INADIMPLENCIA_SCORE'])
dcn['GD_SCORE'] = (dcn['GD_SCORE'] - dcn['GD_SCORE'].min())/(dcn['GD_SCORE'].max() - dcn['GD_SCORE'].min())
dcn['GD_SCORE'] = dcn['GD_SCORE'].fillna(value = 0)

In [414]:
dcn.sort_values('GD_SCORE', ascending=False)

,ESTADO,MUNICIPIO,POP,PIB PER CAPITA,SOL,CASAS,SOLAR_RESIDENCIAL,INDICE DE BUROCRACIA,ICMS,REAIS_POR_KWH,...,PIB_PER_CAPITA_SCORE,SOL_SCORE,CASAS_SCORE,SOLAR_RESIDENCIAL_SCORE,INDICE_DE_BUROCRACIA_SCORE,ICMS_SCORE,REAIS_POR_KWH_SCORE,PRECO(REAIS/M2)_SCORE,INADIMPLENCIA_SCORE,GD_SCORE
2668,MG,Manga,18051,11657.15,6170.0,5116.0,0.008405,58.3,0.30,0.626553,...,0.015583,0.966487,0.001365,0.983645,0.098765,-0.000000,0.820796,0.490833,0.249612,1.000000
2637,MG,Juvenília,5706,8777.22,6142.0,1511.0,0.001985,58.3,0.30,0.626553,...,0.009327,0.959638,0.000356,0.996137,0.098765,-0.000000,0.820796,0.490833,0.249612,0.997064
2687,MG,Matias Cardoso,11360,11127.51,6137.0,2494.0,0.004010,58.3,0.30,0.626553,...,0.014432,0.958415,0.000631,0.992198,0.098765,-0.000000,0.820796,0.490833,0.249612,0.996320
2613,MG,Januária,67958,10446.04,6139.0,17405.0,0.020224,58.3,0.30,0.626553,...,0.012952,0.958904,0.004803,0.960647,0.098765,-0.000000,0.820796,0.490833,0.249612,0.993490
2943,MG,São João das Missões,13232,6428.57,6116.0,2541.0,0.002361,58.3,0.30,0.626553,...,0.004226,0.953278,0.000644,0.995405,0.098765,-0.000000,0.820796,0.490833,0.249612,0.993311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1826,BA,Araçás,12237,19974.72,NaN,3121.0,0.000000,52.5,0.26,0.749490,...,0.033649,NaN,0.000806,1.000000,0.814815,0.258065,0.566819,0.590076,0.195349,0.000000
3312,SP,Biritiba Mirim,33265,23273.12,NaN,8400.0,0.000000,59.1,0.25,0.655414,...,0.040813,NaN,0.002283,1.000000,-0.000000,0.322581,0.761172,0.291103,0.515245,0.000000
2004,BA,Iuiu,11038,9471.45,NaN,2760.0,0.000000,52.5,0.26,0.749490,...,0.010835,NaN,0.000705,1.000000,0.814815,0.258065,0.566819,0.590076,0.195349,0.000000
2068,BA,Muquém do São Francisco,11479,15976.28,NaN,2635.0,0.000000,52.5,0.26,0.749490,...,0.024964,NaN,0.000670,1.000000,0.814815,0.258065,0.566819,0.590076,0.195349,0.000000


In [415]:
cod_mun = pd.read_excel('gdrive/MyDrive/Hacka_Navi/RELATORIO_DTB_BRASIL_MUNICIPIO.xls')
cod_mun = cod_mun.merge(siglas, how = 'left',left_on='UF', right_on='ESTADO')
cod_mun.drop(['UF','ESTADO'], axis = 1, inplace = True)
cod_mun = cod_mun.rename(columns={'SIGLA':'ESTADO'})
cod_mun['ESTADO'] = cod_mun['ESTADO'].str.strip()
cod_mun['COD_MUN'] = cod_mun['COD_MUN'].astype(str)

In [416]:
dcn = dcn.merge(cod_mun, how = 'left',left_on=['ESTADO', 'MUNICIPIO'], right_on=['ESTADO', 'MUNICIPIO'])

In [417]:
dcn.columns

Index(['ESTADO', 'MUNICIPIO', 'POP', 'PIB PER CAPITA', 'SOL', 'CASAS',
       'SOLAR_RESIDENCIAL', 'INDICE DE BUROCRACIA', 'ICMS', 'REAIS_POR_KWH',
       'PRECO(REAIS/M2)', 'INADIMPLENCIA', 'POP_SCORE', 'PIB_PER_CAPITA_SCORE',
       'SOL_SCORE', 'CASAS_SCORE', 'SOLAR_RESIDENCIAL_SCORE',
       'INDICE_DE_BUROCRACIA_SCORE', 'ICMS_SCORE', 'REAIS_POR_KWH_SCORE',
       'PRECO(REAIS/M2)_SCORE', 'INADIMPLENCIA_SCORE', 'GD_SCORE', 'COD_MUN'],
      dtype='object')

In [ ]:
dcn2 = dcn.copy(deep=True)
dcn2.drop(dcn2.columns.difference(['COD_MUN','GD_SCORE']), 1, inplace=True)

In [383]:
dcn.to_csv('gdrive/MyDrive/Hacka_Navi/tabela_consolidada.csv', sep = ';',index = False)

In [560]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [ ]:
#Tratar os dados agora
#dcn = dados consolidados normalizados
dcn = dados_consolidados.copy(deep=True)

dcn['POP_SCORE'] =(dcn['POP']-dcn['POP'].min())/(dcn['POP'].max()-dcn['POP'].min())
dcn['PIB_PER_CAPITA_SCORE'] =(dcn['PIB PER CAPITA']-dcn['PIB PER CAPITA'].min())/(dcn['PIB PER CAPITA'].max()-dcn['PIB PER CAPITA'].min())
dcn['SOL_SCORE'] =(dcn['SOL']-dcn['SOL'].min())/(dcn['SOL'].max()-dcn['SOL'].min())
dcn['CASAS_SCORE'] =(dcn['CASAS']-dcn['CASAS'].min())/(dcn['CASAS'].max()-dcn['CASAS'].min())
dcn['SOLAR_RESIDENCIAL_SCORE'] =(dcn['SOLAR_RESIDENCIAL']-dcn['SOLAR_RESIDENCIAL'].min())/(dcn['SOLAR_RESIDENCIAL'].max()-dcn['SOLAR_RESIDENCIAL'].min())
dcn['INDICE_DE_BUROCRACIA_SCORE'] =(dcn['INDICE DE BUROCRACIA']-dcn['INDICE DE BUROCRACIA'].min())/(dcn['INDICE DE BUROCRACIA'].max()-dcn['INDICE DE BUROCRACIA'].min())
dcn['ICMS_SCORE'] =(dcn['ICMS']-dcn['ICMS'].min())/(dcn['ICMS'].max()-dcn['ICMS'].min())
dcn['REAIS_POR_KWH_SCORE'] =(dcn['REAIS_POR_KWH']-dcn['REAIS_POR_KWH'].min())/(dcn['REAIS_POR_KWH'].max()-dcn['REAIS_POR_KWH'].min())
dcn['PRECO(REAIS/M2)_SCORE'] =(dcn['PRECO(REAIS/M2)']-dcn['PRECO(REAIS/M2)'].min())/(dcn['PRECO(REAIS/M2)'].max()-dcn['PRECO(REAIS/M2)'].min())
dcn['INADIMPLENCIA_SCORE'] =(dcn['INADIMPLENCIA']-dcn['INADIMPLENCIA'].min())/(dcn['INADIMPLENCIA'].max()-dcn['INADIMPLENCIA'].min())

In [559]:
aux = dcn.loc[:,'TOTAL_USINAS':'INADIMPLENCIA_SCORE']
aux = aux.dropna()
y = aux['TOTAL_USINAS']
x = aux.loc[:,'PIB_PER_CAPITA_SCORE':'INADIMPLENCIA_SCORE']

In [561]:
# creating an object of LinearRegression class
lr = LinearRegression()
# fitting the training data
lr.fit(x_train,y_train)
y_prediction =  lr.predict(x_test)
cdf = pd.DataFrame(lr.coef_, x.columns, columns=['Coefficients'])
print(cdf)

                            Coefficients
PIB_PER_CAPITA_SCORE          852.372184
SOL_SCORE                      63.218972
CASAS_SCORE                 13959.396850
SOLAR_RESIDENCIAL_SCORE      2463.099667
INDICE_DE_BUROCRACIA_SCORE     32.913251
ICMS_SCORE                    -52.615328
REAIS_POR_KWH_SCORE           190.860618
PRECO(REAIS/M2)_SCORE         -19.580880
INADIMPLENCIA_SCORE            52.638464
